In [1]:
import os
import uuid

import fire
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.react.agent import ReActAgent
from llama_stack_client.lib.agents.react.tool_parser import ReActOutput
from llama_stack_client.lib.agents.event_logger import EventLogger
from termcolor import colored

# os.environ["TAVILY_API_KEY"] = "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"
base_url = "http://localhost:8321"

client = LlamaStackClient(
    # base_url=f"http://localhost:8321",
    # base_url = "http://localhost:8321"
    base_url=base_url,
    provider_data={"tavily_search_api_key": "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"},
)
available_models = [
    model.identifier for model in client.models.list() if model.model_type == "llm"
]
if not available_models:
    print(colored("No available models. Exiting.", "red"))
    # return

selected_model = available_models[0]
print(colored(f"Using model: {selected_model}", "green"))

# Initialize ReActAgent with just websearch
agent = ReActAgent(
    client=client,
    model=selected_model,
    tools=[
        # "builtin::websearch",
        # "mcp::slack",
        "mcp::pdf",
        # "mcp::crm",
        # "mcp::python",
        ],
    response_format={
        "type": "json_schema",
        "json_schema": ReActOutput.model_json_schema(),
    },
    sampling_params={
        "strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    instructions="You are a helpful assistant",
    tool_config = {
        "tool_choice": "auto",
        "system_message_behavior": "replace",
    },
    max_infer_iters= 2,
)

session_id = agent.create_session(f"react-session-{uuid.uuid4().hex}")

user_prompts = [
    #"What is the capital of Italy?",
    #"Post a slack message in the agentic-ai-slack channel 'C08P4G402HZ' with the result of the question 'What is the capital of Italy?'",
    "Create a pdf with 'Alan Agentic!' as text",
]

for prompt in user_prompts:
    print(colored(f"User> {prompt}", "blue"))
    response = agent.create_turn(
        messages=[{"role": "user", "content": prompt}],
        session_id=session_id,
        stream=True,
    )

    for log in EventLogger().log(response):
        log.print()





Using model: Qwen/Qwen3-14B
User> Create a pdf with 'Alan Agentic!' as text
inference> {


"thought": "The user wants a PDF with 'Alan Agentic!' as text. I need to use the create_pdf_from_markdown function. The function requires markdown content, output filename, paper format, orientation, border, and watermark. The user didn't specify these, so I'll choose defaults. The markdown is just the text 'Alan Agentic!'. Output filename could be 'alan_agentic.pdf'. Paper format defaults to letter, orientation portrait, border 2cm. Watermark isn't specified, so maybe leave it empty or use a default like 'DRAFT'. Wait, the required parameters include watermark, so I need to include it. The user didn't mention a watermark, but the function requires it. Maybe use an empty string or a default. The description says optional watermark text, max 15 characters. So I can set it to an empty string. Let me check the required fields again. The required array includes markdown, outputFilename, paperFormat, 